In [1]:
%cd C:\Users\Tsvetochek\FastFit

import os
from typing import List, Optional, Tuple

import gradio as gr
import numpy as np
import torch
from PIL import Image
from huggingface_hub import snapshot_download

# подключим дополнительные модули
from module.pipeline_fastfit import FastFitPipeline
from parse_utils import DWposeDetector, DensePose, SCHP, multi_ref_cloth_agnostic_mask

PERSON_SIZE = (768, 1024)
DEF_SEED=12345

C:\Users\Tsvetochek\FastFit


In [10]:
%cd C:\Users\Tsvetochek\FastFit
!python ff_gradio.py

C:\Users\Tsvetochek\FastFit


An error occurred while trying to fetch Models/FastFit-MR-1024: Error no file named diffusion_pytorch_model.safetensors found in directory Models/FastFit-MR-1024.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7880): обычно разрешается только одно использование адреса сокета (протокол/сетевой адрес/порт)
Traceback (most recent call last):
  File "C:\Users\Tsvetochek\FastFit\ff_gradio.py", line 367, in <module>
    demo.launch(
  File "C:\ProgramData\anaconda3\envs\fastfit\lib\site-packages\gradio\blocks.py", line 2794, in launch
    ) = http_server.start_server(
  File "C:\ProgramData\anaconda3\envs\fastfit\lib\site-packages\gradio\http_server.py", line 157, in start_server
    raise OSError(
OSError: Cannot find empty port in range: 7880-7880. You can specify a different port by setting the GRADIO_SERVER_PORT environment variable or passing the `server_port` paramete

In [ ]:
jupyter nbconvert --to script gradio.ipynb

In [2]:
# Словарь
# Весь текст, обращенный к пользователю, хранится здесь для удобства управления
translations = {
    "title": "FastFit: Ускорьте виртуальную примерку по нескольким ссылкам",
    "header_html": """
            <div class="main-header">
                <h1 align="center">FastFit: виртуальная примерочная с использованием кэшируемых диффузионных моделей.</h1>            
                <div align="center" style="display: flex; justify-content: center; align-items: center; margin: 20px 0;">
                    <a href="https://github.com/Zheng-Chong/FastFit/blob/main/LICENSE.md" style="margin: 0 2px;"><img src='https://img.shields.io/badge/License-NonCommercial-lightgreen?style=flat&logo=Lisence' alt='License'></a>
                </div>
            </div>
    """,
    "input_settings": "<h3>📤 Виртуальная примерочная</h3>",
    "person_image_header": "<h4>👤 Фотография человека</h4>",
    "person_image_info": '<p class="constraint-text">💡 Рекомендуется загружать изображения в полный рост с соотношением сторон 3:4. Изображения с другим соотношениями сторон будут автоматически обрезаны по центру.</p>',
    "person_image_label": "Загрузите фотографию человека, которого вы хотите переодеть.",
    "ref_garment_header": "<h4>👗 Одежда для примерки</h4>",
    "ref_garment_info": '<p class="constraint-text">⚠️ К сожалению, платья/пальто/комбинезоны нельзя загружать одновременно с другими элементами образа.</p>',
    "upper_body_label": "Рубашки/блузки/футболки/топы и т.п.",
    "lower_body_label": "Брюки/юбки/шорты и т.п.",
    "dress_label": "Платья/Комбинезоны/Пальто",
    "shoes_label": "Обувь",
    "bag_label": "Сумка",
    "generate_params": "<h3>⚙️ Определение параметров</h3>",
    "inference_settings": "<h4>🎮 Настройки вывода</h4>",
    "ref_size_label": "Высота фотографии",
    "ref_size_info": "Задайте высоту фотографии: 512/768/1024，Ширина автоматически рассчитывается для поддержания соотношения сторон 3:4.",
    "steps_label": "Число эпох",
    "steps_info": "Увеличение числа эпок как правило приводит к росту качеству, но снижает скорость генерации.",
    "guidance_label": "CFG Scale",
    "guidance_info": "Контролируйте степень соответствия полученного результата эталонному изображению.",
    "pose_guidance_label": "Учесть позу при генерации",
    "pose_guidance_info": "Использование информации о позе для улучшения качества генерации",
    "generate_button": "🚀 Начать генерацию",
    "status_text": "",
    "result_header": "<h3>📸 Генерация....</h3>",
    "output_image_label": "Сгенерированное изображение",
    "instructions_html": """
        <div class="info-box">
            <h4>💡 Инструкции</h4>
            <ul>
                <li><strong>Требования к фотографии：</strong> Рекомендуется загружать четкую фронтальную фотографию (фронтальный ракурс)  
                            в полный рост с разрешением не менее 768x1024 с соотношением сторон 3:4.</li>
                <li><strong>Обрезка изображения：</strong> Фотография будет автоматически обрезана по центру для сохранени пропорций 3:4，
                при этом какая-то часть изображения может быть утеряна.</li>
                <li><strong>Ограничения примерки：</strong> Платья, комбинезоные нельзя загружать вместе с другими предметами одежды.</li>
                <li><strong>Регулировка параметров：</strong> Размер фотографии влияет на детализацию одежды; 
                снижение числа эпох увеличивает скорость генерации, увеличение числа эпох повышает качество генерации; 
                уменьшение CFG Scale увеличивает свободу модели в генерации, а увелечение - ограничивает</li>
            </ul>
            <h4>⚠️ Меры предосторожности</h4>
            <ul>
                <li>Время генерации зависит от конфигурации оборудования и настроек параметров, пожалуйста, будьте терпеливы.</li>
                <li>Качество итогового изображения зависит от качества исходной фотографии. 
                Рекомендуется использовать высококачественную фотографию для примерки.</li>
            </ul>
        </div>
    """,
    "validation_no_person": "❌ Пожалуйста, загрузите фотографию человека.",
    "validation_dress_conflict": "❌ Платья, комбинезоны, пальто нельзя использовать одновременно с другими элементами одежды.",
    "validation_no_garment": "❌ Пожалуйста, загрузите хотя бы один элемент одежды для примерки.",
    "validation_pass": "✅ Проверка входных данных пройдена",
    "error_model_not_loaded": "❌ Пожалуйста, сначала загрузите модель",
    "error_pose_estimation_failed": "❌ Оценка позы не удалась",
    "error_no_valid_person_image": "❌ Не обнаружено подходящей фотографии человека.",
    "error_generation_failed": "❌ Сборка не удалась: не получилось сгенерировать изображение.",
    "error_exception": "❌ Ошибка сборки: {}",
    "success_generation": "✅ Генерация прошла успешно！",
}

In [3]:
def center_crop_to_aspect_ratio(img: Image.Image, target_ratio: float) -> Image.Image:
    width, height = img.size
    current_ratio = width / height
    
    if current_ratio > target_ratio:
        new_width = int(height * target_ratio)
        new_height = height
        left = (width - new_width) // 2
        top = 0
    else:
        new_width = width
        new_height = int(width / target_ratio)
        left = 0
        top = (height - new_height) // 2
    
    return img.crop((left, top, left + new_width, top + new_height))



In [4]:

class FastFitDemo:
    def __init__(
        self, 
        base_model_path: str = "Models/FastFit-MR-1024", 
        util_model_path: str = "Models/Human-Toolkit",
        mixed_precision: str = "bf16",
        device: str = None,
    ):
        # If not exists, download the models
        if not os.path.exists(base_model_path):
            os.makedirs(base_model_path, exist_ok=True)
            snapshot_download(
                repo_id="zhengchong/FastFit-MR-1024",
                local_dir=base_model_path,
                local_dir_use_symlinks=False
            )
        if not os.path.exists(util_model_path):
            os.makedirs(util_model_path, exist_ok=True)
            snapshot_download(
                repo_id="zhengchong/Human-Toolkit",
                local_dir=util_model_path,
                local_dir_use_symlinks=False
            )
            
        self.device = device if device is not None else "cuda" if torch.cuda.is_available() else "cpu"
        self.dwpose_detector = DWposeDetector(pretrained_model_name_or_path=os.path.join(util_model_path, "DWPose"), device='cpu')
        self.densepose_detector = DensePose(model_path=os.path.join(util_model_path, "DensePose"), device=self.device)
        self.schp_lip_detector = SCHP(ckpt_path=os.path.join(util_model_path, "SCHP", "schp-lip.pth"), device=self.device)
        self.schp_atr_detector = SCHP(ckpt_path=os.path.join(util_model_path, "SCHP", "schp-atr.pth"), device=self.device)
        self.pipeline = FastFitPipeline(
            base_model_path=base_model_path,
            device=self.device,
            mixed_precision=mixed_precision,
            allow_tf32=True
        )

    def validate_inputs(self, person_img, upper_img, lower_img, dress_img, shoe_img, bag_img) -> Tuple[bool, str]:
        # MODIFIED: Simplified check for person_img as it's now a direct Image, not a dict.
        if person_img is None:
            return False, "validation_no_person"
        
        has_upper = upper_img is not None
        has_lower = lower_img is not None
        has_dress = dress_img is not None
        
        if has_dress and (has_upper or has_lower):
            return False, "validation_dress_conflict"
        
        if not (has_dress or has_upper or has_lower or shoe_img or bag_img):
            return False, "validation_no_garment"
        
        return True, "validation_pass"
    
    def preprocess_person_image(self, person_img: Image.Image):
        if self.dwpose_detector is None or self.densepose_detector is None or self.schp_lip_detector is None or self.schp_atr_detector is None:
            raise RuntimeError("Model not initialized")
            
        person_img = person_img.convert("RGB")
        person_img = center_crop_to_aspect_ratio(person_img, 3/4)
        person_img = person_img.resize(PERSON_SIZE, Image.LANCZOS)
        
        pose_img = self.dwpose_detector(person_img)
        if not isinstance(pose_img, Image.Image):
            raise RuntimeError("Pose estimation failed")
        
        densepose_arr = np.array(self.densepose_detector(person_img))
        lip_arr = np.array(self.schp_lip_detector(person_img))
        atr_arr = np.array(self.schp_atr_detector(person_img))
        
        return pose_img, densepose_arr, lip_arr, atr_arr
    
    def generate_mask(self, densepose_arr: np.ndarray, lip_arr: np.ndarray, atr_arr: np.ndarray) -> Image.Image:
        return multi_ref_cloth_agnostic_mask(
            densepose_arr, lip_arr, atr_arr,
            horizon_expand=True
        )
    
    def prepare_reference_images(self, upper_img, lower_img, dress_img, shoe_img, bag_img, ref_height: int) -> Tuple[List[Image.Image], List[str], List[int]]:
        clothing_ref_size = (int(ref_height * 3 / 4), ref_height)
        accessory_ref_size = (384, 512)
        
        ref_images, ref_labels, ref_attention_masks = [], [], []
        
        categories = [
            (upper_img, "upper"), (lower_img, "lower"), (dress_img, "overall"),
            (shoe_img, "shoe"), (bag_img, "bag")
        ]
        
        for img, label in categories:
            target_size = accessory_ref_size if label in ["shoe", "bag"] else clothing_ref_size
            if img is not None:
                img = img.convert("RGB").resize(target_size, Image.LANCZOS)
                ref_images.append(img)
                ref_labels.append(label)
                ref_attention_masks.append(1)
            else:
                ref_images.append(Image.new("RGB", target_size, color=(0, 0, 0)))
                ref_labels.append(label)
                ref_attention_masks.append(0)
        
        return ref_images, ref_labels, ref_attention_masks
    
    def generate_image(
        self, person_img, upper_img, lower_img, dress_img, shoe_img, bag_img,
        ref_height: int, num_inference_steps: int = 50, guidance_scale: float = 2.5,
        enable_pose: bool = True
    ) -> Tuple[Optional[Image.Image], str]:
        
        try:
            is_valid, message_key = self.validate_inputs(person_img, upper_img, lower_img, dress_img, shoe_img, bag_img)
            if not is_valid:
                return None, message_key
            
            if self.pipeline is None:
                return None, "error_model_not_loaded"
            
            # MODIFIED: person_img is now a PIL.Image directly, so no dictionary handling is needed.
            if person_img is None:
                return None, "error_no_valid_person_image"

            processed_person_img = person_img.convert("RGB")
            processed_person_img = center_crop_to_aspect_ratio(processed_person_img, 3/4)
            processed_person_img = processed_person_img.resize(PERSON_SIZE, Image.LANCZOS)
            
            # This function does its own internal processing of the person image
            pose_img, densepose_arr, lip_arr, atr_arr = self.preprocess_person_image(person_img)
            
            # Автоматическая генерация маски
            mask_img = self.generate_mask(densepose_arr, lip_arr, atr_arr)
            
            ref_images, ref_labels, ref_attention_masks = self.prepare_reference_images(
                upper_img, lower_img, dress_img, shoe_img, bag_img, ref_height
            )
            
            generator = torch.Generator(device=self.device).manual_seed(DEF_SEED)
            
            with torch.no_grad():
                result = self.pipeline(
                    person=processed_person_img, mask=mask_img, ref_images=ref_images,
                    ref_labels=ref_labels, ref_attention_masks=ref_attention_masks,
                    pose=pose_img if enable_pose else None, num_inference_steps=num_inference_steps,
                    guidance_scale=guidance_scale, generator=generator, return_pil=True
                )
            
            if isinstance(result, list) and len(result) > 0 and isinstance(result[0], Image.Image):
                return result[0], "success_generation"
            
            return None, "error_generation_failed"
            
        except Exception as e:
            print(f"An exception occurred: {e}")
            return None, f"error_exception:{e}"






In [7]:
def create_demo():
    demo_instance = FastFitDemo()
    
    with gr.Blocks(theme="soft", css="""
        .main-header { text-align: center; margin-bottom: 2rem; }
        .upload-section { border: 2px dashed #e0e0e0; border-radius: 10px; padding: 1rem; margin: 1rem 0; }
        .constraint-text { color: #ff6b6b; font-size: 0.9em; font-style: italic; }
        .info-box { background-color: var(--block-background-fill); border: 1px solid var(--border-color-primary); padding: 1rem; border-radius: 8px; margin-top: 2rem; }
        @media (prefers-color-scheme: light) { .info-box { background-color: #f8f9fa; color: #343a40; border-color: #dee2e6; } }
        @media (prefers-color-scheme: dark) { .info-box { background-color: rgba(255, 255, 255, 0.05); color: #e9ecef; border-color: rgba(255, 255, 255, 0.1); } }
    """) as demo:
        
        
        # --- UI Components ---
        # We define all components here so they can be targeted by the update function
        
        # Header
        header_html = gr.HTML(translations["header_html"])
        
        with gr.Row():
            with gr.Column(scale=1):
                input_settings_header = gr.HTML(translations["input_settings"])
                with gr.Group():
                    person_image_header = gr.HTML(translations["person_image_header"])
                    person_image_info = gr.HTML(translations["person_image_info"])
                    # MODIFIED: Changed from gr.ImageMask to gr.Image
                    person_image = gr.Image(label=translations["person_image_label"], type="pil", sources=["upload"], height=400)
                
                with gr.Group():
                    ref_garment_header = gr.HTML(translations["ref_garment_header"])
                    ref_garment_info = gr.HTML(translations["ref_garment_info"])
                    with gr.Row():
                        upper_image = gr.Image(label=translations["upper_body_label"], type="pil", sources=["upload"], height=200)
                        lower_image = gr.Image(label=translations["lower_body_label"], type="pil", sources=["upload"], height=200)
                    dress_image = gr.Image(label=translations["dress_label"], type="pil", sources=["upload"], height=200)
                    with gr.Row():
                        shoe_image = gr.Image(label=translations["shoes_label"], type="pil", sources=["upload"], height=200)
                        bag_image = gr.Image(label=translations["bag_label"], type="pil", sources=["upload"], height=200)

            
            with gr.Column(scale=1):

                generate_params_header = gr.HTML(translations["generate_params"])
                with gr.Group():
                    inference_settings_header = gr.HTML(translations["inference_settings"])
                    ref_size = gr.Slider(minimum=512, maximum=1024, step=256, value=512, label=translations["ref_size_label"], info=translations["ref_size_info"])
                    num_steps = gr.Slider(minimum=10, maximum=100, value=10, step=1, label=translations["steps_label"], info=translations["steps_info"])
                    guidance_scale = gr.Slider(minimum=1.0, maximum=10.0, value=2.5, step=0.1, label=translations["guidance_label"], info=translations["guidance_info"])
                    with gr.Row():
                        enable_pose = gr.Checkbox(label=translations["pose_guidance_label"], value=True, info=translations["pose_guidance_info"])
                
                generate_btn = gr.Button(translations["generate_button"], variant="primary", size="lg")
                status_text = gr.HTML()
                
                result_header = gr.HTML(translations["result_header"])
                output_image = gr.Image(label=translations["output_image_label"], type="pil", height=400)
        
        instructions_html = gr.HTML(translations["instructions_html"])
        
        # --- Event Handlers ---

        
        
        def generation_wrapper(*args):
            """A wrapper to handle translation of status messages."""
            img, msg_key = demo_instance.generate_image(*args)
            
            t = translations
            
            # Handle exception messages with placeholders
            if "error_exception:" in msg_key:
                key, error_msg = msg_key.split(":", 1)
                status_message = t[key].format(error_msg)
            else:
                status_message = t.get(msg_key, "Unknown status")

            # Add color styling to the status message
            if "✅" in status_message or "successful" in status_message:
                styled_message = f'<p style="color: #51cf66;">{status_message}</p>'
            else:
                styled_message = f'<p style="color: #ff6b6b;">{status_message}</p>'

            return img, styled_message

        generate_btn.click(
            fn=generation_wrapper,
            inputs=[
                person_image, upper_image, lower_image, dress_image, 
                shoe_image, bag_image, ref_size, num_steps, guidance_scale,
                enable_pose
            ],
            outputs=[output_image, status_text]
        )
    
    return demo

In [8]:
if __name__ == "__main__":
    demo = create_demo()
    demo.launch(
        server_name="0.0.0.0",
        server_port=7870,
        share=True,
        show_error=True
    )

An error occurred while trying to fetch Models/FastFit-MR-1024: Error no file named diffusion_pytorch_model.safetensors found in directory Models/FastFit-MR-1024.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


* Running on local URL:  http://0.0.0.0:7870
* Running on public URL: https://255ceb8b75b6c06ca5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


C:\ProgramData\anaconda3\envs\fastfit\lib\site-packages\torch\functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:4316.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [07:55<00:00, 47.53s/it]
